In [ ]:
## load the conceptors code
%run ../python/conceptors.py

In [ ]:
# load the network that we have saved before
net1 = loadObject( "../models/classifier_1_1505307539.311748.pickled" )
net2 = loadObject( "../models/classifier_2_1505309031.43108.pickled" )
net3 = loadObject( "../models/classifier_3_1505308301.813862.pickled" )

# create an initial state for the net
state1 = createState(net1)
state2 = createState(net2)
state3 = createState(net3)

In [ ]:
# net1['C']
# net3['C'].shape
# 128 , 32 , 64

In [ ]:
## load the functions for receiving and sending osc
%run ../python/oscserver.py

inputValues = [ 0, 0, 0 ]

def onInput(ind,f):
    global inputValue
    inputValues[ind] = f
    return True

def onExit():
    print( "exiting" )
    global keepRunning
    keepRunning = False
    oscserver.free()
    return True

oscserver = makeOSCServer(57120, 57401, onExit)
oscserver.onInput = onInput

oscserver.start()

In [ ]:
import time

keepRunning = True

while keepRunning:
    state1 = iterateClassifier(net1, state1, inputValues[0])
    state2 = iterateClassifier(net2, state2, inputValues[1])
    state3 = iterateClassifier(net3, state3, inputValues[2])
#     print( state['x'] )
#     oscserver.send_value( "/output/1", output1[0][0] )
    oscserver.send_array( "/x/CL1", state1['x'].flatten() )
    oscserver.send_array( "/x/CL2", state2['x'].flatten() )
    oscserver.send_array( "/x/CL3", state3['x'].flatten() )
    
    oscserver.send_array( "/evidence/CL1", state1['evidence'] )  
    oscserver.send_array( "/evidence/CL2", state2['evidence'] )  
    oscserver.send_array( "/evidence/CL3", state3['evidence'] )  
    time.sleep(0.033)


In [ ]:
# just in case; free the oscserver
oscserver.free()